In [99]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.colors as mcolors

import glob
from PIL import Image
from scipy import interpolate
from scipy import stats
import matplotlib.ticker as ticker
import warnings
import re

warnings.filterwarnings('ignore')
pd.options.display.float_format = '{:.3f}'.format

In [100]:
project_dir = '/groups/funke/home/mehtay/research/plasticity/'

In [101]:
def clean_coefficients(value):
    value = re.sub(r'[XYWR]0', '', value)
    value = re.sub(r'([XYWR])1', r'\1', value)
    return value

In [131]:
csv_files = glob.glob(project_dir + "/experiments/logs/simdata/recoverability/mlp/exp_*.csv")

dfs = []

for csv_file in csv_files:
    df = pd.read_csv(csv_file)

    dfs.append(df)

df = pd.concat(dfs)

coeffs, other_columns = [], []

for i in range(3):
        for j in range(3):
            for k in range(3):
                for l in range(3):
                    coeffs.append(f"A_{i}{j}{k}{l}")

for column in df.columns:
    if column not in set(coeffs):
        other_columns.append(column)
print(other_columns)
num_epochs = 100

if df["plasticity_model"].unique()[0] == "volterra":
    for coeff in coeffs:
        df[coeff] = pd.to_numeric(df[coeff])
#         df[coeff] = pd.to_numeric(df[coeff], errors='coerce')

df['generation_coeff_init'] = df['generation_coeff_init'].apply(clean_coefficients)
df["moving_avg_window"] = df["moving_avg_window"].astype(int)
df["expid"].unique()

['epoch', 'loss', 'train_time', 'percent_deviance', 'r2_weights', 'r2_activity', 'num_train', 'num_eval', 'num_epochs', 'trials_per_block', 'num_blocks', 'device', 'log_expdata', 'log_mlp_plasticity', 'log_interval', 'use_experimental_data', 'expid', 'fit_data', 'neural_recording_sparsity', 'measurement_noise_scale', 'input_firing_mean', 'input_variance', 'l1_regularization', 'generation_coeff_init', 'generation_model', 'plasticity_coeff_init', 'plasticity_model', 'moving_avg_window', 'data_dir', 'log_dir', 'trainable_coeffs', 'coeff_mask', 'exp_name', 'reward_term', 'layer_sizes']


array([1, 2, 3])

In [132]:
df_subset = df.loc[(df['epoch'] == num_epochs) & (df['reward_term'] == "reward")]
df_subset.head()

,epoch,loss,train_time,percent_deviance,r2_weights,r2_activity,num_train,num_eval,num_epochs,trials_per_block,...,plasticity_coeff_init,plasticity_model,moving_avg_window,data_dir,log_dir,trainable_coeffs,coeff_mask,exp_name,reward_term,layer_sizes
13129,100,0.000,46.917,-0.010,-291350.628,0.369,18,7,100,80,...,random,mlp,10,../data/,logs/,61,NaN,recoverability,reward,"[2, 10, 1]"
13230,100,0.003,47.361,8.742,-1.361,-1.176,18,7,100,80,...,random,mlp,10,../data/,logs/,61,NaN,recoverability,reward,"[2, 10, 1]"
13331,100,0.000,47.208,-0.020,-639378720049.776,-0.456,18,7,100,80,...,random,mlp,10,../data/,logs/,61,NaN,recoverability,reward,"[2, 10, 1]"
13432,100,0.000,47.424,0.000,-5.846,-0.162,18,7,100,80,...,random,mlp,10,../data/,logs/,61,NaN,recoverability,reward,"[2, 10, 1]"
13533,100,0.000,47.482,-0.006,-30123.093,0.593,18,7,100,80,...,random,mlp,10,../data/,logs/,61,NaN,recoverability,reward,"[2, 10, 1]"


In [133]:
df_plot = df_subset[["r2_weights", "r2_activity", "percent_deviance", "generation_coeff_init"]]
# df_plot = df_plot.groupby('generation_coeff_init')[["r2_weights", "r2_activity", "percent_deviance"]].mean()
df_plot = df_plot.groupby('generation_coeff_init')[["r2_weights", "r2_activity", "percent_deviance"]].mean()
# df_plot = df_plot.loc[(df_plot['r2_weights'] > 0)]

In [135]:
df_plot.sort_values(by='percent_deviance', ascending=False, inplace=True)
df_plot[:30]

,r2_weights,r2_activity,percent_deviance
generation_coeff_init,,,
R-0.05XW,-7.769,0.929,88.925
R-0.05YW,-963.039,0.933,88.903
R-0.05XYW,-260.484,0.935,88.897
R-0.05WR,-8.173,0.944,88.895
R-0.05YWR,-98.996,0.944,88.895
R-0.05XWR,-7.686,0.943,88.895
R-0.05XYWR,-38.622,0.944,88.894
R,-6.773,0.943,88.894
R-0.05W,-7.583,0.921,87.558


In [123]:
df_plot.sort_values(by='percent_deviance', ascending=False, inplace=True)
df_plot[:10]

,r2_weights,r2_activity,percent_deviance
generation_coeff_init,,,
XR-0.05XYWR,-0.059,0.563,47.207
XR,0.027,0.552,45.083
XR-0.05XWR,0.052,0.556,44.659
XR-0.05XW,0.157,0.558,43.304
XR-0.05XYW,0.005,0.540,43.255
XR-0.05WR,0.095,0.360,24.209
XR-0.05YW,0.154,0.303,19.152
XR-0.05W,-0.330,0.075,18.778
XR-0.05YWR,-0.157,0.081,7.450
